In [3]:
import os
import re
# import nltk
from zipfile import ZipFile, is_zipfile
from pprint import pprint
# from trie import *
# from index import *
import math

In [4]:
class TrieNode:
    def __init__(self):
        self.children = {}      #Using Hashmap instead of an array because it's more efficient
        self.endOfWord = False  #Indicate if it's the end of a word
        self.locs = []

class Trie:
    def __init__(self, documents=[]):
        self.root = TrieNode() #Initializing root node for the Trie
        
        # Adds all the documents to the trie
        for doc in documents:
            for word in doc.wordsTuples:
                self.insert(word[0], word[1])

    def insert(self, word:str, locs) -> None:
        curr = self.root                            #Start with the root node

        for char in word:                           #For every character in the word
            if char not in curr.children:           #If the character does not exist
                curr.children[char] = TrieNode()    #then make a new node of that character
            curr = curr.children[char]              #Make curr the next character node that already exists or was created
        curr.endOfWord = True                       #Mark that the word ends here

        #Add location
        curr.locs.append(locs)

    def search(self, word:str) -> bool:
        curr = self.root                    #Start with the root node

        for char in word:                   #For every character in the word
            if char not in curr.children:   #If the character does not exist
                return False                #The word does not exist
            curr = curr.children[char]      #Make curr the next character node if that character exists
        return curr.endOfWord               #The word exists

    def prefixExists(self, prefix: str) -> bool:
        curr = self.root                        #Start with the root node

        for char in prefix:                     #For every character in the word
            if char not in curr.children:       #If the character does not exist
                return False                    #The prefix does not exist
            curr = curr.children[char]          #Make curr the next character node if that character exists
        return True                             #The prefix exists

    def _prefixSearchHelper(self, word, curr):
        if curr.endOfWord:      #If there is a delimeter node then append the word to the hashmap
            self._prefixWords[word] = curr.locs

        for char, node in curr.children.items():
            self._prefixSearchHelper(word + char, node) 

    def prefixSearch(self, prefix: str) -> list:
        curr = self.root                        #Start with the root node

        for char in prefix:                     #For every character in the word
            if char not in curr.children:       #If the character does not exist
                return []                       #The prefix does not exist
            curr = curr.children[char]          #Make curr the next character node if that character exists

        self._prefixWords = {}                  #An empty hashmap for the list of words returneed by the prefix search
        self._prefixSearchHelper(prefix, curr)

        return self._prefixWords


In [5]:
class Document:
    def __init__(self, docId, data):
        self.docId = docId
        self.data = data
        
        self._tokenize()

    def _tokenize(self):
        self.words = []
        self.wordsDict = {}
        self.wordsTuples = []
        for word in re.finditer(r'\S+', self.data):
            #append word to the list of words in the document
            self.words.append(word.group(0))

            # To generate dictionary in the format {Word: [(Document ID, Starting Index, Ending Index)]..}
            if self.wordsDict.get(word.group(0)) == None:
                self.wordsDict[word.group(0)] = [(word.start(), word.end())]
            else:
                self.wordsDict[word.group(0)].append((word.start(), word.end()))
            
            # To generate list of words in the format (Word, (Document ID, Starting Index, Ending Index))
            wordTuple = [word.group(0), ((self.docId, word.start(), word.end()))]
            self.wordsTuples.append(wordTuple)

In [6]:
class InvertedIndex:
    def __init__(self, documents=[]) -> None:
        self.index = {}
        self.documents = documents
        self.totalDocuments = len(documents)

        # Adds all the documents to the index
        for doc in documents:
            self._insertDoc(doc)

    
    def _insertDoc(self, doc): 
        for key in doc.wordsDict:
            if key not in self.index:
                self.index[key] = [(doc.docId, len(doc.wordsDict[key]))]
            else:
                self.index[key].append((doc.docId, len(doc.wordsDict[key])))

        
    def query(self, query: str, desiredResults: int):
        query = query.split()

        # finding tf for query
        tf = [[0 for col in range(self.totalDocuments)] for row in range(len(query))]

        for doc in range(self.totalDocuments):
            for word in range(len(query)):
                tf[word][doc] = self._findOccurences(query[word], self.documents[doc].docId) / len(self.documents[doc].words)

        
        # finding idf for query
        idf = [math.log2(self.totalDocuments / len(self.index[word])) for word in query]

        # finding tf-idf for the query now
        tf_idf = [[0 for col in range(self.totalDocuments)] for row in range(len(query))]

        for doc in range(self.totalDocuments):
            for word in range(len(query)):
                tf_idf[word][doc] = tf[word][doc] * idf[word]

        #compute sum of the found tf-idf
        final_tf_idf = [0 for col in range(self.totalDocuments)]

        for doc in range(self.totalDocuments):
            for word in range(len(query)):
                final_tf_idf[doc] += tf_idf[word][doc]

        #computing rank now (AAAA i didnt store the document ID and now im v lazy to change the structure so pls don't mind what i'm about to do)
        ranks = []
        for i in range(len(final_tf_idf)):
            if final_tf_idf[i] != 0:
                ranks.append((final_tf_idf[i], self.documents[i].docId))
        ranks.sort(reverse=True)
        
        finalRanks = []

        for i in range(1, len(ranks) + 1):
            finalRanks.append((i, ranks[i-1][1]))

        return finalRanks
                

    def _findOccurences(self, word, docId):
        occurences = self.index[word]

        for occurence in occurences:
            if occurence[0] == docId:
                return occurence[1]
                
        return 0





In [7]:
class Corpus:
    def __init__(self, path) -> None:
        
        self.documents = []
        
        if is_zipfile(path):
            file = ZipFile(path, mode = 'r')
            print(file)
            filePaths = file.namelist()
            file.extractall() 
        elif os.path.isdir(path):
            #files = os.listdir(path)
            filePaths = list()
            for (dirpath, dirnames, filenames) in os.walk(path):
                filePaths += [os.path.join(dirpath, file) for file in filenames]
        else:
            filePaths = [path]
        
        for filePath in filePaths:
            if os.path.isfile(filePath):
                with open(filePath, 'r', errors = 'ignore') as f:

                    text = f.read().lower()
                    newtext = re.sub(r'[^\w\s]', '', text)

                doc = Document(filePath, newtext)
                # print(filePath)
                self.documents.append(doc)

        self.trie = Trie(self.documents)  #Creates a Trie object from the corpus
        
        self.index = InvertedIndex(self.documents)

    def prefix_complete(self, query:str):
        '''
            Turn the query into lowercase since every word in the Corpus is lowercase
            Then call prefix_complete method of the trie object of class Trie to get a list of prefix matches
            Return the list of Prefix Matches
        '''
        query = query.lower()
        return self.trie.prefixSearch(query)

    def query(self, query: str, desiredResults: int):
        return self.index.query(query, desiredResults)

In [8]:
corpus = Corpus("corpus.zip")

<zipfile.ZipFile filename='corpus.zip' mode='r'>


In [10]:
pprint(corpus.prefix_complete('return'))

{'return': [('1-10/6-10/10', 9344, 9350),
            ('1-10/6-10/10', 19408, 19414),
            ('1-10/6-10/10', 30469, 30475),
            ('1-10/1-5/4.txt', 6501, 6507),
            ('1-10/1-5/3.txt', 5739, 5745),
            ('1-10/1-5/3.txt', 7314, 7320),
            ('1-10/1-5/2.txt', 8, 14),
            ('1-10/1-5/2.txt', 15069, 15075),
            ('1-10/1-5/1.txt', 4734, 4740),
            ('1-10/1-5/1.txt', 26610, 26616),
            ('11-15/14.txt', 4652, 4658),
            ('11-15/14.txt', 27747, 27753),
            ('11-15/14.txt', 29322, 29328),
            ('11-15/13.txt', 3409, 3415),
            ('11-15/13.txt', 5490, 5496),
            ('11-15/13.txt', 10348, 10354),
            ('11-15/13.txt', 17055, 17061),
            ('11-15/12.txt', 1689, 1695),
            ('11-15/12.txt', 13416, 13422)],
 'returned': [('1-10/6-10/9', 8566, 8574),
              ('1-10/6-10/9', 13674, 13682),
              ('1-10/6-10/9', 17100, 17108),
              ('1-10/6-10/9', 19094, 1910

In [191]:
print('hen'.split())

['hen']


In [291]:
import pathlib

path = 'corpus/'
filePaths = list()
for (dirpath, dirnames, filenames) in os.walk(path):
    filePaths += [os.path.join(dirpath, file) for file in filenames]

print(filePaths)

import pathlib

filePaths = [pathlib.PureWindowsPath(p).as_posix() for p in filePaths]
print(filePaths)


# for i in range(len(filePaths)):
#     filePaths[i] = str(filePaths[i][len(path):]).replace(f'\\\\', '/')
#     # print(filePaths[i])

# for i in range(len(filePaths)):
#     print(filePaths[i])

# print(filePaths)


['corpus/16', 'corpus/17', 'corpus/18', 'corpus/1-10\\1-5\\1.txt', 'corpus/1-10\\1-5\\2.txt', 'corpus/1-10\\1-5\\3.txt', 'corpus/1-10\\1-5\\4.txt', 'corpus/1-10\\1-5\\5.txt', 'corpus/1-10\\6-10\\10', 'corpus/1-10\\6-10\\6', 'corpus/1-10\\6-10\\7', 'corpus/1-10\\6-10\\8', 'corpus/1-10\\6-10\\9', 'corpus/11-15\\11.txt', 'corpus/11-15\\12.txt', 'corpus/11-15\\13.txt', 'corpus/11-15\\14.txt', 'corpus/11-15\\15.txt']
corpus/16
corpus/17
corpus/18
corpus/1-10/1-5/1.txt
corpus/1-10/1-5/2.txt
corpus/1-10/1-5/3.txt
corpus/1-10/1-5/4.txt
corpus/1-10/1-5/5.txt
corpus/1-10/6-10/10
corpus/1-10/6-10/6
corpus/1-10/6-10/7
corpus/1-10/6-10/8
corpus/1-10/6-10/9
corpus/11-15/11.txt
corpus/11-15/12.txt
corpus/11-15/13.txt
corpus/11-15/14.txt
corpus/11-15/15.txt
